In [178]:
import re
import time
from itertools import chain
from itertools import repeat
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

#if traceid in id1 then ok
#if traceid in id2 then force velocity -> negative and id1 <-> id2    type1<->type2     unchange     dist  force==magnitutde of f  p4=magnitutde of ft
traceid = 68940
rp = 0.000415
column_name_id1 = 'c_p_idtype_trace[1]'
column_name_id2 = 'c_p_idtype_trace[2]'
column_name_type1 = 'c_p_idtype_trace[3]'
column_name_type2 = 'c_p_idtype_trace[4]'
column_change_sign = ['fjinx', 'fjiny', 'fjinz', 'fjitx', 'fjity', 'fjitz', 'vijnx', 'vijny', 'vijnz', 'vijtx', 'vijty', 'vijtz']

runfolder = '0101_mu05_freeze'
readpath = '/home/ic6413/mylammps/myfiles/run/' + runfolder + '/output/python/'
filename_custom = 'custom.h5'
filename_pair = 'pair.h5'
outputpath = '/home/ic6413/mylammps/myfiles/run/' + runfolder + '/output/python/'

start = time.process_time()

datapdc = pd.read_hdf( readpath + filename_custom, 'datapd')
datapdc['r'] = 0.9*rp*(datapdc['type']==1) + 1.0*rp*(datapdc['type']==2) + 1.1*rp*(datapdc['type']==3)

datapdci = datapdc.rename(columns = {'id':'id_i', 'r':'ri'} )
datapdci = datapdci.loc[ datapdci['id_i'] == traceid]

datapdcj = datapdc.rename(columns = {'id':'id_j', 'r':'rj', 'x':'xj', 'y':'yj', 'z':'zj', 'vx':'vjx', 'vy':'vjy', 'vz':'vjz', 'omegax':'omegajx', 'omegay':'omegajy', 'omegaz':'omegajz'} )
datapdcj = datapdcj[['step', 'rj','id_j', 'xj', 'yj', 'zj', 'vjx', 'vjy', 'vjz', 'omegajx', 'omegajy', 'omegajz'] ]

datapdp = pd.read_hdf(readpath + filename_pair, 'datapd')
id_change_sign = (datapdp[column_name_id2] == traceid)
datapdp.loc[id_change_sign,[column_name_id1, column_name_id2]] = datapdp.loc[id_change_sign,[column_name_id2, column_name_id1]].values
datapdp.loc[id_change_sign,[column_name_type1, column_name_type2]] = datapdp.loc[id_change_sign,[column_name_type2, column_name_type1]].values
datapdp.loc[id_change_sign,column_change_sign] = -datapdp.loc[id_change_sign,column_change_sign].values
datapdp = datapdp.rename(columns = {'c_p_idtype_trace[1]':'id_i', 'c_p_idtype_trace[2]':'id_j', 'c_p_idtype_trace[3]':'type_i', 'c_p_idtype_trace[4]':'type_j'})


datapdcip =  pd.merge(datapdci, datapdp, how='inner', on=['step', 'id_i'])
datapdcipcj =  pd.merge(datapdcip, datapdcj, how='inner', on=['step', 'id_j'])

end = time.process_time()
print(end-start)
datapdcipcj.to_hdf(outputpath + 'dump_c' + str(traceid) + 'pcj' + '.h5',key='datapd') 

9.672618069999999


In [179]:
datapd = pd.read_hdf(readpath + 'dump_c' + str(traceid) + 'pcj' + '.h5', 'datapd')
list(datapd)

['step',
 'id_i',
 'type',
 'v_EKP_atom',
 'v_KEtr_atom',
 'c_KEt_atom',
 'c_KEr_atom',
 'x',
 'y',
 'z',
 'vx',
 'vy',
 'vz',
 'fx',
 'fy',
 'fz',
 'omegax',
 'omegay',
 'omegaz',
 'tqx',
 'tqy',
 'tqz',
 'f_force_all[1]',
 'f_force_all[2]',
 'f_force_all[3]',
 'f_force_pair[1]',
 'f_force_pair[2]',
 'f_force_pair[3]',
 'c_n_contact',
 'v_check_nb_all',
 'ri',
 'index',
 'id_j',
 'type_i',
 'type_j',
 'distance',
 'fjinx',
 'fjiny',
 'fjinz',
 'fjin',
 'fjitx',
 'fjity',
 'fjitz',
 'fjit',
 'vijnx',
 'vijny',
 'vijnz',
 'vijtx',
 'vijty',
 'vijtz',
 'rj',
 'xj',
 'yj',
 'zj',
 'vjx',
 'vjy',
 'vjz',
 'omegajx',
 'omegajy',
 'omegajz']

In [194]:
ri = datapd['ri'].values
rj = datapd['rj'].values

datapd['mi'] = 4/3*np.pi*datapd['ri']**3*2500
datapd['mj'] = 4/3*np.pi*datapd['rj']**3*2500
datapd['meff'] = datapd['mi'].values * datapd['mj'].values/ ( datapd['mi'].values + datapd['mj'].values )

datapd['xij'] = datapd['x'].values-datapd['xj'].values
datapd['yij'] = datapd['y'].values-datapd['yj'].values
datapd['zij'] = datapd['z'].values-datapd['zj'].values

datapd['fjix'] = datapd['fjinx'].values+datapd['fjitx'].values
datapd['fjiy'] = datapd['fjiny'].values+datapd['fjity'].values
datapd['fjiz'] = datapd['fjinz'].values+datapd['fjitz'].values


rij_vector = np.stack([datapd['xij'].values,datapd['yij'].values,datapd['zij'].values], axis=-1)
rij = (np.sum(rij_vector*rij_vector, axis=1))**0.5
rij_unit = rij_vector/np.array([rij,rij,rij]).T
 

rji_vector = -rij_vector
rji_unit = -rij_unit

ri_vector = np.einsum('ij,i->ij',rji_unit,ri) 
rj_vector = np.einsum('ij,i->ij',rij_unit,rj)  
datapd['overlapij'] = rij - (datapd['ri'].values + datapd['rj'].values)

#check v consistency           
v_vector = np.stack((datapd['vx'].values, datapd['vy'].values, datapd['vz'].values), axis=-1)
vn = np.einsum('ij,ij->i', v_vector, rij_unit)
vn_vector = np.einsum('i,ij->ij',vn, rij_unit)
vt_vector = v_vector-vn_vector

vj_vector = np.stack((datapd['vjx'].values,datapd['vjy'].values,datapd['vjz'].values), axis=-1)
vjn = np.einsum('ij,ij->i', vj_vector, rij_unit)
vjn_vector = np.einsum('i,ij->ij',vjn, rij_unit)
vjt_vector = vj_vector-vjn_vector

vij_vector = v_vector-vj_vector

vijn_vector = np.stack((datapd['vijnx'].values,datapd['vijny'].values,datapd['vijnz'].values), axis=-1)
vijt_vector = np.stack((datapd['vijtx'].values,datapd['vijty'].values,datapd['vijtz'].values), axis=-1)

fji_vector = np.stack([datapd['fjix'].values,datapd['fjiy'].values,datapd['fjiz'].values], axis=-1)
fjin_vector = np.stack([datapd['fjinx'].values,datapd['fjiny'].values,datapd['fjinz'].values], axis=-1)
fjit_vector = np.stack([datapd['fjitx'].values,datapd['fjity'].values,datapd['fjitz'].values], axis=-1)

fjin = np.sqrt(np.einsum('ij,ij->i', fjin_vector, fjin_vector) )
fjit = np.sqrt(np.einsum('ij,ij->i', fjit_vector, fjit_vector) )


ox = datapd['omegax'].values
oy = datapd['omegay'].values
oz = datapd['omegaz'].values
o_vector = np.stack([ox,oy,oz], axis=-1)
omegar_vector = np.cross(o_vector,ri_vector)
vc_vector = omegar_vector + v_vector
vct_vector = omegar_vector + vt_vector

ojx = datapd['omegajx'].values
ojy = datapd['omegajy'].values
ojz = datapd['omegajz'].values
oj_vector = np.stack([ojx,ojy,ojz], axis=-1)
omegarj_vector = np.cross(oj_vector,rj_vector)
vjc_vector = omegarj_vector + vj_vector
vjct_vector = omegarj_vector + vjt_vector


vijc_vector = vc_vector - vjc_vector

vijct_vector = vct_vector - vjct_vector

check1 = vij_vector - vijt_vector - vijn_vector

p = rij_vector
q = vc_vector   #v_vector-vj_vector-vijt_vector-vijn_vector

costheta0 = np.einsum('ij,ij->i', p, q) / np.sqrt((np.einsum('ij,ij->i', p, p) * np.einsum('ij,ij->i', q, q) ) )
qn = np.einsum('ij,ij->i', q, rij_unit)
qn_vector = np.einsum('i,ij->ij',qn, rij_unit)
qt_vector = q - qn_vector

#check vi-vj = vij
check_v = vij_vector-vijn_vector-vijt_vector
#check sum fji = f
datapd_sumf_calculate = datapd.groupby(['step'])['fjix', 'fjiy', 'fjiz'].sum()
datapd_sumf = datapd.groupby(['step'])['fx', 'fy', 'fz'].mean()
checkf = np.array(datapd_sumf) - np.array(datapd_sumf_calculate)

norm = np.sqrt(np.einsum('ij,ij->i', check1, check1) )

In [200]:
#check torque
tqji_vector = np.cross(ri_vector,fjit_vector)
datapd['tqjix'] = tqji_vector[:,0]
datapd['tqjiy'] = tqji_vector[:,1]
datapd['tqjiz'] = tqji_vector[:,2]

datapd_sumtq_calculate = datapd.groupby(['step'])['tqjix', 'tqjiy', 'tqjiz'].sum()
datapd_sumtq = datapd.groupby(['step'])['tqx', 'tqy', 'tqz'].mean()
checktq = np.array(datapd_sumtq) - np.array(datapd_sumtq_calculate)
checktq

array([[ 2.06603024e-17, -9.65082033e-18,  4.52785366e-17],
       [-2.15441094e-17,  8.92490723e-17, -1.99135018e-16],
       [ 8.51989402e-17,  4.66492725e-17,  4.33826870e-17],
       ...,
       [-1.61257869e-17, -1.53898184e-18,  2.39960201e-17],
       [-1.11978002e-16,  5.19357692e-17, -2.29904172e-16],
       [-3.06585508e-16,  3.46762564e-16, -2.38345235e-16]])

In [208]:
#check vjit parallel to fjit
p = vij_vector
q = fjit_vector 
costheta4 = np.einsum('ij,ij->i', p, q) / np.sqrt((np.einsum('ij,ij->i', p, p) * np.einsum('ij,ij->i', q, q) ) )
costheta4

array([-0.0043294 , -0.00432955, -0.00433036, ..., -0.06800702,
       -0.06801325, -0.06801367])

In [278]:
#check a v x
ax = datapd['fx'].values/datapd['mi'].values
ay = datapd['fy'].values/datapd['mi'].values
az = datapd['fz'].values/datapd['mi'].values
a_vector = np.stack((ax, ay, az), axis=-1)
a_vector_part = 2*10**-6*a_vector[2:-2]
vdiff = v_vector[4:] - v_vector[0:-4]
check1 = vdiff-a_vector_part

x = datapd['x'].values
y = datapd['y'].values
z = datapd['z'].values
coord_vector = np.stack((x, y, z), axis=-1)
coord_diff = coord_vector[4:] - coord_vector[0:-4]
v_vector_part = 2*10**-6*v_vector[2:-2]

check2 = coord_diff-v_vector_part

check2/coord_diff

array([[-1.35353841e-08,  1.37457476e-08,  5.60667243e-07],
       [-1.35353841e-08,  1.37457476e-08,  5.60667243e-07],
       [ 5.35861798e-08, -6.43864109e-08, -2.40266796e-05],
       ...,
       [-9.55402866e-10, -3.16356038e-09,  1.22414514e-08],
       [ 6.47317810e-10, -2.85992649e-10,  3.60979848e-08],
       [ 6.47317810e-10, -2.85992649e-10,  3.60979848e-08]])

In [277]:
#check falling speed

vij_vector

array([[-4.15950332e-05,  4.15950330e-05, -1.18858533e-07],
       [-4.15950332e-05,  4.15950330e-05, -1.18858533e-07],
       [-4.15988803e-05,  4.15988800e-05, -1.18880533e-07],
       ...,
       [-6.48748990e-04,  6.48748977e-04, -2.89385901e-05],
       [-6.48808925e-04,  6.48808912e-04, -2.89439420e-05],
       [-6.48808925e-04,  6.48808912e-04, -2.89439420e-05]])

In [202]:
mu = 0.5
kn = 8837.32631448687 
kt = 2524.95037556768 
gamma_n = 34445.5603308471 
gamma_t = 17222.7801654236
meff = datapd['meff'].values
overlapij = datapd['overlapij'].values
fjin_vector_calculate = -kn * np.array([overlapij,overlapij,overlapij]).T*rij_unit  - gamma_n * np.array([meff,meff,meff]).T *vijn_vector 
check_f = fjin_vector_calculate - fjin_vector
print (fjin_vector_calculate)
print (fjin_vector)
print (check_f)
p = rij_vector
q = fjin_vector_calculate 
costheta4 = np.einsum('ij,ij->i', p, q) / np.sqrt((np.einsum('ij,ij->i', p, p) * np.einsum('ij,ij->i', q, q) ) )
#qn = np.einsum('ij,ij->i', q, rij_unit)
fjin_vector

[[ 3.06928720e-06  3.07437538e-06  1.78067202e-06]
 [-3.08731817e-06 -3.08189044e-06  1.89950358e-06]
 [ 3.06928697e-06  3.07437561e-06  1.78067202e-06]
 ...
 [-3.13615766e-06 -3.05122169e-06  1.90415531e-06]
 [ 3.04122100e-06  3.12085491e-06  1.78512465e-06]
 [-3.13616330e-06 -3.05121943e-06  1.90415618e-06]]
[[ 3.06929e-06  3.07438e-06  1.78067e-06]
 [-3.08732e-06 -3.08189e-06  1.89950e-06]
 [ 3.06929e-06  3.07438e-06  1.78067e-06]
 ...
 [-3.13616e-06 -3.05122e-06  1.90416e-06]
 [ 3.04122e-06  3.12086e-06  1.78513e-06]
 [-3.13616e-06 -3.05122e-06  1.90416e-06]]
[[-2.79583184e-12 -4.62493957e-12  2.01825742e-12]
 [ 1.83458135e-12 -4.39970382e-13  3.58061817e-12]
 [-3.02878942e-12 -4.38729523e-12  2.01894341e-12]
 ...
 [ 2.34211409e-12 -1.68565230e-12 -4.69198536e-12]
 [ 1.00333995e-12 -5.08809678e-12 -5.34996369e-12]
 [-3.30222747e-12  5.69228872e-13 -3.82403793e-12]]


array([[ 3.06929e-06,  3.07438e-06,  1.78067e-06],
       [-3.08732e-06, -3.08189e-06,  1.89950e-06],
       [ 3.06929e-06,  3.07438e-06,  1.78067e-06],
       ...,
       [-3.13616e-06, -3.05122e-06,  1.90416e-06],
       [ 3.04122e-06,  3.12086e-06,  1.78513e-06],
       [-3.13616e-06, -3.05122e-06,  1.90416e-06]])

In [229]:
#check tangent damp part
fjit_vector_calculate_damp_part = - gamma_t * np.array([meff,meff,meff]).T *vijct_vector 

fjit_vector_calculate_damp_part/fjit_vector

array([[-1.43540625e-06,  1.44822076e-06,  1.69524846e-08],
       [ 1.24650362e-06, -1.22256790e-06,  2.07136278e-08],
       [-1.43519301e-06,  1.44869872e-06,  1.73021236e-08],
       ...,
       [ 2.15295793e-05, -1.64220469e-05,  4.67584676e-06],
       [-2.00175105e-05,  2.44895766e-05,  4.80640559e-06],
       [ 2.15314108e-05, -1.64237631e-05,  4.67631520e-06]])

In [123]:
rij_unit

array([[ 0.65373417,  0.65481791,  0.37926921],
       [-0.64888133, -0.64774055,  0.39923077],
       [ 0.65373412,  0.65481796,  0.37926921],
       ...,
       [-0.65721049, -0.63941138,  0.39903314],
       [ 0.64582068,  0.6627314 ,  0.37908144],
       [-0.65721132, -0.63941056,  0.39903311]])

In [121]:
fjit_vector

array([[-2.41114e-07, -2.44262e-07,  8.37324e-07],
       [ 2.57046e-07,  2.53876e-07,  8.29690e-07],
       [-2.41113e-07, -2.44262e-07,  8.37324e-07],
       ...,
       [ 2.78033e-07,  2.28285e-07,  8.23727e-07],
       [-2.15797e-07, -2.65213e-07,  8.31301e-07],
       [ 2.78035e-07,  2.28282e-07,  8.23726e-07]])

In [81]:
rij_vector

array([[ 0.0005426 ,  0.0005435 ,  0.00031479],
       [-0.00051164, -0.00051074,  0.00031479],
       [ 0.0005426 ,  0.0005435 ,  0.00031479],
       ...,
       [-0.00051821, -0.00050418,  0.00031464],
       [ 0.00053603,  0.00055007,  0.00031464],
       [-0.00051821, -0.00050417,  0.00031464]])

In [118]:
datapd.iloc[1:100, list(range(14,15)) + list(range(35,45) ) + list(range(63,70)) ]
datapd.iloc[:, list(range(10,30))  ]

,step,id_i,type,v_EKP_atom,v_KEtr_atom,c_KEt_atom,c_KEr_atom,x,y,z,...,tqy,tqz,f_force_all[1],f_force_all[2],f_force_all[3],f_force_pair[1],f_force_pair[2],f_force_pair[3],c_n_contact,v_check_nb_all
0,270290.0,68940.0,1.0,3.435948e-15,3.435948e-15,9.440286e-16,2.491919e-15,-0.015271,0.027954,0.023965,...,-8.054969e-13,1.234432e-14,-2.099012e-09,2.098954e-09,-1.200369e-11,-2.099012e-09,2.098954e-09,0.000005,2.0,1.0
1,270290.0,68940.0,1.0,3.435948e-15,3.435948e-15,9.440286e-16,2.491919e-15,-0.015271,0.027954,0.023965,...,-8.054969e-13,1.234432e-14,-2.099012e-09,2.098954e-09,-1.200369e-11,-2.099012e-09,2.098954e-09,0.000005,2.0,1.0
2,270291.0,68940.0,1.0,3.436583e-15,3.436583e-15,9.442032e-16,2.492380e-15,-0.015271,0.027954,0.023965,...,-8.055716e-13,1.234547e-14,-2.099192e-09,2.099162e-09,-1.200468e-11,-2.099192e-09,2.099162e-09,0.000005,2.0,1.0
3,270291.0,68940.0,1.0,3.436583e-15,3.436583e-15,9.442032e-16,2.492380e-15,-0.015271,0.027954,0.023965,...,-8.055716e-13,1.234547e-14,-2.099192e-09,2.099162e-09,-1.200468e-11,-2.099192e-09,2.099162e-09,0.000005,2.0,1.0
4,270292.0,68940.0,1.0,3.437219e-15,3.437219e-15,9.443779e-16,2.492841e-15,-0.015271,0.027954,0.023965,...,-8.056463e-13,1.234661e-14,-2.099372e-09,2.099370e-09,-1.200903e-11,-2.099372e-09,2.099370e-09,0.000005,2.0,1.0
5,270292.0,68940.0,1.0,3.437219e-15,3.437219e-15,9.443779e-16,2.492841e-15,-0.015271,0.027954,0.023965,...,-8.056463e-13,1.234661e-14,-2.099372e-09,2.099370e-09,-1.200903e-11,-2.099372e-09,2.099370e-09,0.000005,2.0,1.0
6,270293.0,68940.0,1.0,3.437855e-15,3.437855e-15,9.445526e-16,2.493302e-15,-0.015271,0.027954,0.023965,...,-8.057207e-13,1.234775e-14,-2.099554e-09,2.099577e-09,-1.201507e-11,-2.099554e-09,2.099577e-09,0.000005,2.0,1.0
7,270293.0,68940.0,1.0,3.437855e-15,3.437855e-15,9.445526e-16,2.493302e-15,-0.015271,0.027954,0.023965,...,-8.057207e-13,1.234775e-14,-2.099554e-09,2.099577e-09,-1.201507e-11,-2.099554e-09,2.099577e-09,0.000005,2.0,1.0
8,270294.0,68940.0,1.0,3.438491e-15,3.438491e-15,9.447273e-16,2.493764e-15,-0.015271,0.027954,0.023965,...,-8.057950e-13,1.234889e-14,-2.099736e-09,2.099783e-09,-1.201564e-11,-2.099736e-09,2.099783e-09,0.000005,2.0,1.0
9,270294.0,68940.0,1.0,3.438491e-15,3.438491e-15,9.447273e-16,2.493764e-15,-0.015271,0.027954,0.023965,...,-8.057950e-13,1.234889e-14,-2.099736e-09,2.099783e-09,-1.201564e-11,-2.099736e-09,2.099783e-09,0.000005,2.0,1.0


In [119]:
fjin_vector

array([[ 3.06929e-06,  3.07438e-06,  1.78067e-06],
       [-3.08732e-06, -3.08189e-06,  1.89950e-06],
       [ 3.06929e-06,  3.07438e-06,  1.78067e-06],
       ...,
       [-3.13616e-06, -3.05122e-06,  1.90416e-06],
       [ 3.04122e-06,  3.12086e-06,  1.78513e-06],
       [-3.13616e-06, -3.05122e-06,  1.90416e-06]])

In [832]:
overlapij


array([-3.60193304e-08,  2.17193474e-08,  7.94543740e-08, ...,
        7.94543740e-08,  7.94543740e-08,  1.37185750e-07])

In [814]:
datapd_test = datapd[['step','id_j','x','y','z','xj','yj','zj','vx','fx','overlapij','fjinx','fjiny','fjinz','fjitx','fjity','fjitz']]
d_datapd = datapd_test.diff(periods=2)
datapd_test = datapd_test.assign(dx=d_datapd['x'].values,dvx=d_datapd['vx'].values,dfx=d_datapd['fx'].values)
#datapd_test['dx'],datapd_test['dvx'],datapd_test['dfx'] = datapd_test.diff().values
datapd_test['x']

0       0.028992
1       0.028992
2       0.028992
3       0.028992
4       0.028992
5       0.028992
6       0.028992
7       0.028992
8       0.028992
9       0.028992
10      0.028992
11      0.028992
12      0.028992
13      0.028992
14      0.028992
15      0.028992
16      0.028992
17      0.028992
18      0.028992
19      0.028992
20      0.028992
21      0.028992
22      0.028992
23      0.028992
24      0.028992
25      0.028992
26      0.028992
27      0.028992
28      0.028992
29      0.028992
          ...   
1320    0.028992
1321    0.028992
1322    0.028992
1323    0.028992
1324    0.028992
1325    0.028992
1326    0.028992
1327    0.028992
1328    0.028992
1329    0.028992
1330    0.028992
1331    0.028992
1332    0.028992
1333    0.028992
1334    0.028992
1335    0.028992
1336    0.028992
1337    0.028992
1338    0.028992
1339    0.028992
1340    0.028992
1341    0.028992
1342    0.028992
1343    0.028992
1344    0.028992
1345    0.028992
1346    0.028992
1347    0.0289

In [486]:
datapd[['step','id_j','x','y','z','xj','yj','zj','vx','overlapij','fjinx','fjiny','fjinz']]

,step,id_j,x,y,z,xj,yj,zj,vx,overlapij,fjinx,fjiny,fjinz
0,1.0,6773.0,-0.026356,-0.026356,0.023650,-0.025899,-0.025833,0.023908,0.000000e+00,-6.104420e-06,-0.033076,-3.782840e-02,-0.018693
1,1.0,6773.0,-0.025526,-0.026240,0.024528,-0.025899,-0.025833,0.023908,-4.649760e-10,-6.762926e-08,0.000001,-1.360330e-06,0.000002
2,1.0,6775.0,-0.025302,-0.025302,0.023650,-0.024810,-0.025829,0.024139,0.000000e+00,5.411192e-08,-0.000002,1.779400e-06,-0.000002
3,1.0,6775.0,-0.025526,-0.026240,0.024528,-0.024810,-0.025829,0.024139,-4.649760e-10,1.188389e-09,-0.000008,-4.873250e-06,0.000005
4,1.0,6974.0,-0.025302,-0.025302,0.023650,-0.025623,-0.025292,0.024370,0.000000e+00,-2.128120e-08,0.000002,-6.562720e-08,-0.000005
5,1.0,6748.0,-0.025526,-0.026240,0.024528,-0.025843,-0.026869,0.024014,-4.649760e-10,1.376935e-09,0.000005,9.033910e-06,0.000007
6,286.0,6773.0,-0.026356,-0.026356,0.023650,-0.025899,-0.025833,0.023908,0.000000e+00,-6.104420e-06,-0.033077,-3.782930e-02,-0.018694
7,286.0,6773.0,-0.025526,-0.026240,0.024528,-0.025899,-0.025833,0.023908,-4.668310e-10,-6.762926e-08,0.000001,-1.360330e-06,0.000002
8,286.0,6775.0,-0.025302,-0.025302,0.023650,-0.024810,-0.025829,0.024139,0.000000e+00,5.411192e-08,-0.000002,1.779400e-06,-0.000002
9,286.0,6775.0,-0.025526,-0.026240,0.024528,-0.024810,-0.025829,0.024139,-4.668310e-10,1.188389e-09,-0.000008,-4.873250e-06,0.000005


In [774]:
rij-0.0004565*2

array([-3.60193304e-08,  2.17193474e-08,  7.94543740e-08, ...,
        7.94543740e-08,  7.94543740e-08,  1.37185750e-07])

In [801]:
rj[1]-0.0004565


5.421010862427522e-20

In [649]:
0.00083*29

0.02407

In [663]:
(0.0230896**2+ 0.0209338**2)**0.5/0.00083

37.550055828167615